---

# Analise partidas League of Legends

League of Legends (*LoL*) é um jogo eletrônico online do gênero de batalha entre multijogadores.

O mapa possui 02 lados, cada time (de 05 jogadores) inicia em um dos lados. Cada time tem o objetivo de destruir o **nexus** do time adversário.

Até atingir o objetivo-principal, ambos os times se enfrentam pelo mapa, principalmente pelas 03 rotas (superior, meio e inferior), estas que possuem **torres** que defendem os avanços eminentes do time adversário.

Há uma série de objetivos paralelos que vão auxiliando os times a adquirir dinheiro, consequentemente melhorando seus equipamentos, para atingir o objetivo principal.


## Grupo:

* Lucas Picollo - 16204446
* Rafael Parola - 15200940
* Rian Bruggemann - 15101200

---

## Descrição

O projeto consiste na análise de mais de 100 mil partidas de League of Legends, dos elos Master, GrandMaster e Challenger do servidor da Korea em 2020.

O dataset foi retirado do [Kaggle](https://www.kaggle.com/) e pode ser encontrado [neste link](https://www.kaggle.com/gyejr95/league-of-legendslol-ranked-games-2020-ver1).

Pelo tamanho do conjunto de dados e por possuir colunas em formato json, o criador do dataset disponibilizou os arquivos em formato .pickle, porem as analises serao feitas atraves do arquivos csv.

---

## Importações





### Base de dados

In [ ]:
from google.colab import files

files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d gyejr95/league-of-legendslol-ranked-games-2020-ver1
!unzip league-of-legendslol-ranked-games-2020-ver1.zip data

### Aplicação

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools
import json

%matplotlib inline

In [7]:
df_matches = pd.read_csv('data/match_data_version1.csv')
df_matches.head()

,Unnamed: 0,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,status.message,status.status_code
0,0,1.585155e+12,1323.0,4.247263e+09,CLASSIC,MATCHED_GAME,10.6.314.4405,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
1,1,1.585152e+12,1317.0,4.247156e+09,CLASSIC,MATCHED_GAME,10.6.314.4405,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
2,2,1.585059e+12,932.0,4.243963e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
3,3,1.584978e+12,2098.0,4.241678e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
4,4,1.584973e+12,2344.0,4.241539e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN


## Entendimento do Negócio


### Dataset jogadores por partidas

#### Analisando as dimensoes do dataset

In [8]:
df_matches.shape

(108829, 15)

#### Agrupando as partidas por modo de jogo, podemos perceber a presenca de diferentes modos de jogo:

In [ ]:
df_matches.groupby('gameMode').size()

Para a análise iremos utilizar apenas dos jogos do **modo clássico**, pois os diferentes modos de jogo oferecem dinâmicas distintas entre eles, então removeremos os demais. Posteriormente removeremos os demais modos da base de dados.

Como temos em mão um dataframe serializado, ao converter para um json, acarretava em um json **não válido**, então tivemos que tratar as aspas simples ( ' ) para aspas duplas ( " )

Eh possivel perceber tambem que a coluna que representa o resultado da partida eh uma coluna textual contendo 'win' ou 'lose'

In [ ]:
df_matches['participants'].iloc[0]

Precisaremos função para substituir os valores booleanos da coluna "Win" juntamento com a troca de aspas simples para duplas, para entao conseguirmos parsear os registros para um hash.


Percebemos que para cada jogador eh gerado um id interno sendo ele um numero de 1 a 10, o que nao eh valido para identificarmos um jogador entro os demais do servidor. Para isso, precisaremos de um identificador global, que se encontra na colunda participantIdentities. 

Precisaremos de um meio de atrelar as estatisticas de um jogador em determinada partida com um o seu identificador global do servidor.


In [ ]:
df_matches['participants'].head()

### Dataset de estatisticas das partida

Alem da base de dados de jogos, possuimos dois datasets contendo estatisticas de cada partida, dividos entre vitorias e derrotas.**bold text**

In [ ]:
df_winners = pd.read_csv('match_winner_data_version1.csv')
df_winners.info()

Analisando a tabela de informacões sobre o time vencedor, podemos extrair as seguintes informações:

* A coluna que diz se um time foi vencedor ou não apresenta dados em formato de **string**, devemos converte-lo para **bool**
* As colunas **dominionVictoryScore** e **bans** nao serão relevantes para obtenção dos resultados esperados.

In [ ]:
df_winners.head()

In [ ]:
df_winners.info()

#### Explicacao por coluna:

* teamId : identificador do time (100,200)
* win : resultado da partida (Win,False)
* firstBlood : primeira morte (boolean)
* firstTower : primeira torre (boolean)
* firstTower : primeiro inibidor (boolean)
* firstTower : primeiro baron (boolean)
* firstTower : primeiro dragon (boolean)
* firstTower : primeiro arauto (boolean)
* towerKills : torres destruida (integer)
* inhibitorKills : inibidores destruidos (integer)
* baronKills : barons feitos (integer)
* dragonKills : dragoes feitos (integer)
* riftHeraldKills : arautos feitos (integer)
* bans : campeoes banidos (json)
* gameId : gameId (foreign key)



Analisando a tabela de informacões sobre o time perdedor, podemos extrair as seguintes informações:

* A coluna que diz se um time foi vencedor ou não apresenta dados em formato de **string**, devemos converte-lo para **bool**
* As colunas **dominionVictoryScore** e **bans** nao serão relevantes para obtenção dos resultados esperados.

Possuimos dois arquivos diferentes com a mesma estrutura, um para os dados do **time vencedor** e outro para o **perdedor**.

Faremos a junção dos dois datasets fazendo a distrinção dos dados através da coluna **win**.

In [ ]:
df_loosers = pd.read_csv('match_loser_data_version1.csv')
df_winners = pd.read_csv('match_winner_data_version1.csv')
df_winners.head()

---

### Objetivos

Com a analise dos dados finalizada, temos como output aos objetivos da analise, sao eles:



#### Quais campeões possuem mais counters pick?

*Counter pick* => significa que um campeão X possui desvantagens naturais (do jogo) em relação ao campeão Y, sendo desta forma, menos eficiente em batalha.

Feito uma seleção nas colunas que usaremos para analise:

* gameId
* gameDuration - tempo de duracao da partida
* gameMode - modo de jogo
* participantIdentities - identificadores do jogadores
* participants - dados sobre o jogador durante a partida

Deve ser ressaltado que existem colunas que contém dados em formato **json**, o que requer um **passo adicional** de conversão para fazermos as analises.




#### Existem jogadores que são counters de outros jogadores?

Buscaremos nos dados a relacao entre jogaores especificos para encontrar se ha alguma relacao entre eles diretamente, analisando numero de vitorias individuais.


#### Quais são as caracteristicas da partida que são mais relevantes para a decisão do time vencedor?

Encontrar as features que sao mais correlacionadas com o resultado da partida.

Com essa analise feita queremos poder, dado uma partida sem o resultado explicitado, dizer qual será o time vencedor.


## Preparação dos Dados

### Dataset de estatisticas de partidas

Selecao das colunas relevantes para a analise do dataset de partidas.

In [ ]:
columns = ['gameId', 'gameMode', 'gameDuration', 'participantIdentities', 'participants']

df_matches = pd.read_csv('match_data_version1.csv', usecols=columns)
df_matches.head()

#### Remocao dos demais modos de jogo alem do Classico.

In [ ]:
df_matches.drop(df_matches[df_matches['gameMode']!= 'CLASSIC'].index, axis=0, inplace=True)
df_matches.groupby('gameMode').size()

#### Parser da coluna 'participants' e 'participantsIdentities'

In [ ]:
df_matches['participants'] = df_matches['participants'].apply(lambda x: x.replace('False', 'false').replace('True', 'true').replace('\'', '\"'), axis=1)
df_matches['participants'] = df_matches['participants'].apply(lambda x: json.loads(x))
df_matches['participants'].head()

In [ ]:
df_matches['participantIdentities'] = df_matches['participantIdentities'].apply(lambda x: json.loads( x.replace('False', 'false').replace('True', 'true').replace('\'', '\"')))
df_matches['participantIdentities'].head()

####  Merge de ID com estatisticas
Faremos o merge das colunas atribuindo o ID global a cada jogador e dividindo os times entre ganhadores e perdedores.

In [ ]:
def merge_identity_with_stats(participant,stats):
  return {
    **stats['stats'], 
    'lane': stats['timeline']['lane'], 
    'role': stats['timeline']['role'], 
    'championId': stats['championId'], 
    "summonerName": participant["player"]["summonerName"]
  }

xs = []

# for _, match in itertools.islice(df_matches.iterrows(), 1):
for _, match in df_matches.iterrows():
  y = {
      'game': match['gameId'],
      'win_stats': {},
      'lose_stats': {}
  }
  for stats, identity in zip(match['participants'], match['participantIdentities']):
    y['win_stats' if stats['stats']['win'] else 'lose_stats'][identity["player"]["summonerName"]] = {
        **stats['stats'], 
      'lane': stats['timeline']['lane'], 
      'role': stats['timeline']['role'], 
      'championId': stats['championId'],
    }
  xs.append(y)

df_game_stats = pd.DataFrame(xs)
df_game_stats.head()


---
### Dataset de vitorias e derrotas

Merge do dataset de vitorias e derrotas, e diminuicao do numero de colunas, selecionando as coluans mais relvantes.

In [ ]:
columns = [
 'win','firstBlood','firstTower','firstInhibitor','firstBaron',
 'firstDragon','firstRiftHerald','towerKills','inhibitorKills','baronKills',
 'dragonKills','vilemawKills','riftHeraldKills','gameId'
]

df_loosers = pd.read_csv('match_loser_data_version1.csv', usecols=columns)
df_winners = pd.read_csv('match_winner_data_version1.csv', usecols=columns)

# Juncao dos datasets
df_results = pd.concat([df_loosers, df_winners])

# Conversao da coluna win para boolean
df_results['win'] = df_results['win'].map(lambda r: r == 'Win')
df_results.head()

#####  Correlacao entre as colunas

Com a visualizacao do mapa de calor (heatmap) eh possivel vermos quais as variaveis sao mais relevantes para sabermos o resultado da partida.

In [ ]:
corr = df_results.corr()

ax = sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

ax.set_title("Attributes Correlation Matrix", fontsize = 16)

Podemos observar uma forte correlação com a vitória na partida:
1.   towerKills
2.   riftHeraldKills
3.   firstInhibitor
4.   dragonKills



##### Bar chart informativos entre correlações x wins

Com essa visualizaco eh possivel vermos a correlacao entre as principais 

In [ ]:
width = 0.35  # the width of the bars

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# tower/wins
axs[0,0].bar(df_results['towerKills'].unique(), df_results.groupby('towerKills').sum('win')['win'])
axs[0,0].set_xlabel("Torres")
axs[0,0].set_ylabel("Vitorias")

# dragon/wins
axs[0,1].bar(df_results['dragonKills'].unique(), df_results.groupby('dragonKills').sum('win')['win'])
axs[0,1].set_xlabel("Dragoes")
axs[0,1].set_ylabel("Vitorias")

# herald/wins
axs[1,0].bar(df_results['riftHeraldKills'].unique(), df_results.groupby('riftHeraldKills').sum('win')['win'])
axs[1,0].set_xlabel("Arautos")
axs[1,0].set_ylabel("Vitorias")

# baron/wins
axs[1,1].bar(df_results['firstInhibitor'].unique(), df_results.groupby('firstInhibitor').sum('win')['win'])
axs[1,1].set_xlabel("Inibidores")
axs[1,1].set_ylabel("Vitorias")

plt.show()



**Towers x Wins**
- A equipe que destruiu mais torres, consequentemente apresenta melhores resultados (vitórias)
- Uma observação válida para esta situação é que ao destruir 01 torre, se equipara com a proporção de destruir 11. 

- A provavel explicação para essa situação é que a equipe que estava em desvantagem no jogo, e provavelmente havia perdido sua 1ª torre há pouco, rendeu-se, desta forma, finalizando o jogo e dando a vitória pro outro time.

**DragonKills x Wins**
- A equipe que atingir a marca de 3 dragons no jogo, tem 5 vezes mais chance de ganhar. 

- Uma observação de por que 4 dragons ter uma chance menor de a de 3, pode ser devido a estratégia e velocidade do jogo (não sendo possível fazer um 4º dragon, pois o jogo finalizou antes disso ou não proporcionou momentos oportunos para que a equipe faça o 4º dragon)

**Heralds x Wins**

- Este é o item que melhor proporciona uma indicação de qual equipe ganhará o jogo, pois trata-se de um 'side objective' do jogo, que sendo bem controlado por uma equipe, rendeu de 4 a 6 vezes mais vitórias.

**Barons x Wins**

- Nesta comparação podemos observar a ausência de importância do objetivo "matar o Baron" dentro da partida. Desta forma, equipes que se empenham em realizar esse objetivo, tendem a perder as partidas.
